In [1]:
import pandas as pd
twitter = pd.read_csv("labeled_tweet_table.csv")
twitter

,Tweet,Name,Screen Name,Description,Lang,img_path,Race,Under 21
0,"YKAR, a futuristic sans serif font by @Emmeran...",Chad Boyce,djsnipa1,"Multimedia Developer, Graphic Designer, DJ, an...",NaN,profile pics/60147.jpeg,4.0,0
1,@MBonvoyAssist Who can I contact about the ver...,Chad Boyce,djsnipa1,"Multimedia Developer, Graphic Designer, DJ, an...",NaN,profile pics/60147.jpeg,4.0,0
2,@SSlnes I’d like to win!,Chad Boyce,djsnipa1,"Multimedia Developer, Graphic Designer, DJ, an...",NaN,profile pics/60147.jpeg,4.0,0
3,@LuckyDucksNFT @Rydog,Chad Boyce,djsnipa1,"Multimedia Developer, Graphic Designer, DJ, an...",NaN,profile pics/60147.jpeg,4.0,0
4,Now I'm heading to B1000th Floor! #quickrogue,Chad Boyce,djsnipa1,"Multimedia Developer, Graphic Designer, DJ, an...",NaN,profile pics/60147.jpeg,4.0,0
...,...,...,...,...,...,...,...,...
313468,Preach it 🙌🙌 https://t.co/8xPvSZpiLb,✿♥☆✧աaʄʄʟɛ զʊɛɛռ✧☆♥✿,landry_rosalee,other acount- @furious_waffles and @i_cant_wor...,NaN,profile pics/64232.jpeg,NaN,0
313469,Wut? https://t.co/FxR7WlXhDz,✿♥☆✧աaʄʄʟɛ զʊɛɛռ✧☆♥✿,landry_rosalee,other acount- @furious_waffles and @i_cant_wor...,NaN,profile pics/64232.jpeg,NaN,0
313470,"Im not trying to be rude, im just saying that ...",✿♥☆✧աaʄʄʟɛ զʊɛɛռ✧☆♥✿,landry_rosalee,other acount- @furious_waffles and @i_cant_wor...,NaN,profile pics/64232.jpeg,NaN,0
313471,"Sir, I'd like to know why you're being so rude...",✿♥☆✧աaʄʄʟɛ զʊɛɛռ✧☆♥✿,landry_rosalee,other acount- @furious_waffles and @i_cant_wor...,NaN,profile pics/64232.jpeg,NaN,0


In [4]:
import re
regexMap={r"<[\w'/'\s]*>": "",r"[\'\"\-]+": "",r"@[\w]+":"",r"http\S+" : ""}
def preprocess(datainput):
    t=datainput
    for regx in regexMap.keys():
        t = re.sub(regx, regexMap[regx], t)
    return t

def labelFilter(label):
  return int(label)

twitter=twitter[["Tweet","Race"]]
#twitter["Tweet"]=twitter["Tweet"].apply(preprocess)
twitter = twitter.dropna()
twitter["Race"] = twitter["Race"].apply(labelFilter)
#twitter["Tweet"]
twitter

,Tweet,Race
0,"YKAR, a futuristic sans serif font by @Emmeran...",4
1,@MBonvoyAssist Who can I contact about the ver...,4
2,@SSlnes I’d like to win!,4
3,@LuckyDucksNFT @Rydog,4
4,Now I'm heading to B1000th Floor! #quickrogue,4
...,...,...
313398,I'm raising money for Help Feed the Homeless. ...,4
313399,I'm raising money for Help Feed the Homeless. ...,4
313400,I'm raising money for Help Feed the Homeless. ...,4
313401,I'm raising money for Help Feed the Homeless. ...,4


In [5]:
twitter = twitter[twitter["Race"] != 5]
twitter["Race"] -= 1
twitter["Race"].value_counts()

4    242886
1     28719
2     17797
3      9885
Name: Race, dtype: int64

In [3]:
#!pip install tensorflow_text==2.5.0.
import tensorflow as tf
import tensorflow.keras as keras
import tensorflow_hub as hub
import tensorflow_text as text

In [4]:
bert_preprocess = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
bert_encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4")

sentences = ["I love you", "I hate you"]
preprocessed_text = bert_preprocess(sentences)
bert_encoder(preprocessed_text)["pooled_output"]

<tf.Tensor: shape=(2, 768), dtype=float32, numpy=
array([[-9.1058314e-01, -3.5070494e-01, -3.2318351e-01, ...,
        -4.5160763e-05, -7.1246177e-01,  9.1118807e-01],
       [-8.1203640e-01, -2.2017559e-01,  3.5842064e-01, ...,
         4.2030013e-01, -5.8162081e-01,  8.2115543e-01]], dtype=float32)>

In [5]:
data, labels = twitter["Tweet"], twitter["Race"]
data.shape, labels.shape

((310584,), (310584,))

In [8]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(data, labels, test_size=.2)

In [9]:
text_input = keras.layers.Input(shape=(), dtype=tf.string)
preprocessed_text = bert_preprocess(text_input)
outputs = bert_encoder(preprocessed_text)

dense = keras.layers.Dense(4, activation='softmax')(outputs["pooled_output"])

model = keras.Model(inputs=[text_input], outputs=[dense])
model.summary()
model.compile(optimizer='adam', loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None,)]            0                                            
__________________________________________________________________________________________________
keras_layer (KerasLayer)        {'input_type_ids': ( 0           input_2[0][0]                    
__________________________________________________________________________________________________
keras_layer_1 (KerasLayer)      {'sequence_output':  109482241   keras_layer[1][0]                
                                                                 keras_layer[1][1]                
                                                                 keras_layer[1][2]                
____________________________________________________________________________________________

In [ ]:
model.fit(x_train, y_train, epochs=5)



Epoch 1/5
3445/7765 [============>.................] - ETA: 1:33:10 - loss: 0.8164 - accuracy: 0.7812

In [ ]:
from sklearn.metrics import accuracy_score

y_pred = np.argmax(model.predict(x_test), axis=1)
accuracy_score(y_pred, y_test)